In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, GRU
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, r2_score
import keras.backend as K
import tensorflow as tf

class NeurogenesisLayer(tf.keras.layers.Layer):
    def __init__(self, num_neurons):
        super(NeurogenesisLayer, self).__init__()
        self.num_neurons = num_neurons

    def build(self, input_shape):
        super(NeurogenesisLayer, self).build(input_shape)
        self.neurons = tf.keras.layers.Dense(self.num_neurons)

    def call(self, inputs):
        new_neurons = self.neurons(inputs)
        print("Criando novos neurônios!")
        neurons = tf.concat([inputs, new_neurons], axis=1)
        return neurons


def perda_penalidade_repeticao(y_true, y_pred):
    diff = y_pred[:, 1:] - y_pred[:, :-1]
    penalidade = K.sum(K.square(diff))
    perda_mse = K.mean(K.square(y_true - y_pred))
    perda_total = perda_mse + penalidade

    return perda_total

def perda_personalizada(y_true, y_pred):
    limite_superior = 2.0
    perda_penalizacao = K.mean(K.maximum(y_pred - limite_superior, 0))
    return perda_penalizacao



df = pd.read_csv('https://raw.githubusercontent.com/cardosource/OraculoAI/main/atualizado.csv', header=None)


data = np.array(df).reshape(-1, 1)

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

#LSTM model
lookback = 6
X = []
y = []
for i in range(lookback, len(data)):
    X.append(data_scaled[i-lookback:i, 0])
    y.append(data_scaled[i, 0])
X, y = np.array(X), np.array(y)


train_size = int(len(X) * 0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


model_lstm = Sequential()
model_lstm.add(GRU(50, activation='relu', input_shape=(lookback, 1)))
model_lstm.add(Dropout(0.2))
model_lstm.add(NeurogenesisLayer(11))  # Adicione a camada de neurogênese com 10 novos neurônios
model_lstm.add(Dense(1))

initial_learning_rate = 0.001
model_lstm.compile(loss=perda_personalizada, optimizer='adam')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001)

# Treino LSTM model
model_lstm.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)),
               y_train,
               epochs=200,
               verbose=0,
               callbacks=[reduce_lr],
               validation_data=(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test))


forecast_lstm = []
current_lstm = X_test[-1]
for i in range(36):
    x_input = np.reshape(current_lstm, (1, lookback, 1))
    yhat = model_lstm.predict(x_input, verbose=0)
    forecast_lstm.append(yhat[0])
    current_lstm = np.append(current_lstm[1:], yhat[0])
forecast_lstm = scaler.inverse_transform(np.array(forecast_lstm).reshape(-1, 1))



model1 = Sequential()
model1.add(GRU(50, activation='relu', input_shape=(lookback, 1)))
model1.add(Dropout(0.2))
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')

model2 = Sequential()
model2.add(GRU(100, activation='relu', input_shape=(lookback, 1)))
model2.add(Dropout(0.2))
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam')

model1.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=100, verbose=0)
model2.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=100, verbose=0)


forecast_ensemble = []
current_ensemble = X_test[-1]
for i in range(36):
    x_input = np.reshape(current_ensemble, (1, lookback, 1))
    yhat1 = model1.predict(x_input, verbose=0)
    yhat2 = model2.predict(x_input, verbose=0)
    yhat_ensemble = np.mean([yhat1[0], yhat2[0]])  # Combine predictions with mean
    forecast_ensemble.append(yhat_ensemble)
    current_ensemble = np.append(current_ensemble[1:], yhat_ensemble)
forecast_ensemble = scaler.inverse_transform(np.array(forecast_ensemble).reshape(-1, 1))




ensemble_predictions = np.mean([model1.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1))),
                               model2.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)))]
                               , axis=0)
mse_ensemble = mean_squared_error(y_test, ensemble_predictions.flatten())  #converter  a matriz multidimensional para um vetor unidimensional
r2_ensemble = r2_score(y_test, ensemble_predictions.flatten())   
print('Erro Quadrático Médio:', mse_ensemble)
print('Coeficiente de Determinação:', r2_ensemble)

forecast_ensemble = forecast_ensemble.flatten()
forecast_ensemble = forecast_ensemble.reshape(-1, 6)   
forecast_ensemble = np.round(forecast_ensemble, 0)

print(forecast_ensemble)













2024-04-10 14:05:11.362828: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 14:05:13.773352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 14:05:13.773802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 14:05:14.092966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 14:05:15.007359: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 14:05:29.092765: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

Criando novos neurônios!
Criando novos neurônios!
Criando novos neurônios!
Criando novos neurônios!
Criando novos neurônios!
7/7 [==============================] - 0s 14ms/step
Erro Quadrático Médio: 0.07538006754752988
Coeficiente de Determinação: 0.9240910166457702
[[40. 48. 10. 18. 25. 34.]
 [42. 48. 12. 19. 26. 34.]
 [42. 48. 14. 20. 27. 35.]
 [43. 48. 16. 21. 27. 35.]
 [43. 48. 17. 22. 28. 36.]
 [43. 47. 19. 23. 29. 36.]]
